In [ ]:
'''
Breaks the large raw tweet data file into manageable portions

'''

import pandas as pd

chunk_size = 10000
batch_no = 1

print("Beginning to parse tweets...")

for chunk in pd.read_csv("01-03_covid_data.csv", chunksize = chunk_size, error_bad_lines=False):
        chunk.to_csv("covid_data" + str(batch_no) + ".csv", index = False)
        batch_no += 1

print("Finished parsing tweets.")

In [8]:
'''
Creates a dictionary mapping the value of "state" in each tweet with the average sentiment

senitiment.polarity: denotes the sentiment of a text, float from -1 (completely negative) to 1 (completely positive) 

'''

import csv
import pandas as pd
import textblob
import glob

sentimentDict = {}
tweetList = []
path = "./*.csv"

print("Beginning to assign sentiments...")
#I put the samller chunked tweet files in to a directory called revisedCovidData
for filename in glob.glob(path):
    #Iterate through this directory, reading each file
    with open(filename, 'r', encoding="utf-8") as rawTweets:
        #open as CSV iterator
        readCSV = csv.reader(rawTweets)
        #the first file has the header line, which needs to be skipped
        #if filename == ".\covid_data1.csv":
            #next(readCSV)
        #Iterate through individual tweets
        for line in readCSV:
            if line[9] != "Null" or "us_state":
                #calls text of each tweet as a TextBlob object
                text = textblob.TextBlob(line[1])
                #line[9] = state; if this state is already in the dictionary, the sentiment gets averaged
                if line[9] in sentimentDict:
                    sentimentDict[line[9]] = (sentimentDict[line[9]] +  (text.sentiment.polarity))/2
                    #sentimentDict[line[9]].append(text.sentiment.polarity) <= if wanting a list instead of an average
                else:
                    #it state not already accounted for, saves it as a key with the relevant sentiment
                    sentimentDict.update({line[9]: (text.sentiment.polarity)})


#denote finished. These are large files, it takes a while
print("Finished assigning sentiments.")

Beginning to assign sentiments
Finished assigning sentiments.


In [11]:
print("writing sentiments to file...")
with open("twitter_sentiments.csv", "w") as outFile:
    writer = csv.writer(outFile)
    writer.writerow(["State", "Avg. Sentiment"])
    for key, value in sentimentDict.items():
        writer.writerow([key, value])
print("finsished writing to file")